In [7]:
import pandas as pd

df = pd.read_csv('..\data\data.csv')
print(df.columns.tolist())

['data_donor_email', 'session_id', 'version', 'event_time', 'A_KEY_TIME_MEAN', 'BOTH_DEV_CC_COUNT', 'BOTH_DEV_C_COUNT', 'BOTH_DEV_M_COUNT', 'CLUSTERED_KEYS', 'CLUSTERED_NR_KEYS', 'CLUSTERED_R_KEYS', 'DELETE_KEY', 'DELETE_KEYS', 'GAP_D1', 'GAP_D2', 'GAP_D3', 'GAP_D4', 'GAP_D5', 'GAP_D6', 'GAP_D7', 'GAP_D8', 'GAP_D9', 'GAP_SD', 'KEY_COUNT', 'KEY_GAP_TIME_D1', 'KEY_GAP_TIME_D2', 'KEY_GAP_TIME_D3', 'KEY_GAP_TIME_D4', 'KEY_GAP_TIME_D5', 'KEY_GAP_TIME_D6', 'KEY_GAP_TIME_D7', 'KEY_GAP_TIME_D8', 'KEY_GAP_TIME_D9', 'KEY_TIME', 'KEY_TIME_D1', 'KEY_TIME_D2', 'KEY_TIME_D3', 'KEY_TIME_D4', 'KEY_TIME_D5', 'KEY_TIME_D6', 'KEY_TIME_D7', 'KEY_TIME_D8', 'KEY_TIME_D9', 'KEY_TIME_MEAN', 'MANY_KEYS_BY_KEY_PRESSED', 'MOUSE_CLICK_RATIO', 'MOUSE_COUNT', 'MOUSE_DDIR_D1', 'MOUSE_DDIR_D2', 'MOUSE_DDIR_D3', 'MOUSE_DDIR_D4', 'MOUSE_DDIR_D5', 'MOUSE_DDIR_D6', 'MOUSE_DDIR_D7', 'MOUSE_DDIR_D8', 'MOUSE_DIR_D1', 'MOUSE_DIR_D2', 'MOUSE_DIR_D3', 'MOUSE_DIR_D4', 'MOUSE_DIR_D5', 'MOUSE_DIR_D6', 'MOUSE_DIR_D7', 'MOUSE_DIR_D

In [13]:
import time

import pandas as pd
from pandas import DataFrame
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm


def remove_na_values(df: DataFrame, drop_column_threshold=0.15):
    null_var = df.isnull().sum() / df.shape[0]
    drop_columns = null_var[null_var > drop_column_threshold].keys()
    df = df.drop(columns=drop_columns)
    df = df.dropna()
    return df


def train_models(models: list, df: DataFrame):
    results = pd.DataFrame(columns=['model', 'acc', 'tpr'])
    for model in tqdm(models):
        x = df.iloc[:, 4:]
        y = df.iloc[:, 0]

        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

        model.fit(x_train, y_train)
        # y_pred_proba = model.predict_proba(x_test)
        y_pred = model.predict(x_test)

        # auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
        acc = accuracy_score(y_test, y_pred)
        tpr = recall_score(y_test, y_pred, average='micro')

        results = results.append({'model': str(model), 'acc': acc, 'tpr': tpr}, ignore_index=True)
    return results, models


In [81]:
import re

path_to_data = 'C:\\Users\\smatuszewski_adm\\Desktop\\PW\\4sem\\DSW\\DataScienceWorkshop2023L\\data\\data.csv'

df = remove_na_values(pd.read_csv(path_to_data))

column_names = df.columns[4:]

def get_group(s):
    pattern = re.compile(r"(.+?)_D[0-9]")
    search = pattern.search(s)
    if search is None:
        return s
    return search.group(1)

feature_groups = np.array([get_group(s) for s in column_names])

feature_groups_map = {s: np.where(feature_groups == s)[0] for s in np.unique(feature_groups)}


In [104]:
def remove_columns_ints(df: DataFrame, exclude_columns: list):
    columns_to_remove = [x + 4 for x in exclude_columns]
    result = df.drop(df.columns[columns_to_remove], axis=1)
    return result

In [103]:
def remain_columns_ints(df: DataFrame, remain_columns: list):
    columns_to_remain = [0,1,2,3] + [x + 4 for x in remain_columns]
    result = df.iloc[:, columns_to_remain]
    return result

In [16]:
feature_groups

array(['A_KEY_TIME_MEAN', 'BOTH_DEV_C_COUNT', 'GAP', 'GAP', 'GAP', 'GAP',
       'GAP', 'GAP', 'GAP', 'GAP', 'GAP', 'GAP_SD', 'KEY_COUNT',
       'KEY_GAP_TIME', 'KEY_GAP_TIME', 'KEY_GAP_TIME', 'KEY_GAP_TIME',
       'KEY_GAP_TIME', 'KEY_GAP_TIME', 'KEY_GAP_TIME', 'KEY_GAP_TIME',
       'KEY_GAP_TIME', 'KEY_TIME', 'KEY_TIME', 'KEY_TIME', 'KEY_TIME',
       'KEY_TIME', 'KEY_TIME', 'KEY_TIME', 'KEY_TIME', 'KEY_TIME',
       'KEY_TIME', 'KEY_TIME_MEAN', 'MOUSE_COUNT', 'MOUSE_DDIR',
       'MOUSE_DDIR', 'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR',
       'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR',
       'MOUSE_DIST', 'MOUSE_DIST', 'MOUSE_DIST', 'MOUSE_DIST',
       'MOUSE_DIST', 'MOUSE_DIST', 'MOUSE_DIST', 'MOUSE_DIST',
       'MOUSE_DIST', 'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP',
       'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP',
       'MOUSE_SPEED', 'MOUSE_SPEED', 'MOUSE_SPEED', 'MOUSE_SPEED',
       'MOUSE_SPEED', 'MOUSE_SPEE

In [105]:
models = [
    # KNeighborsClassifier(3, weights="distance"),
    # GaussianNB(),
    # BernoulliNB(),
    # ComplementNB(),
    # MultinomialNB(),
    RandomForestClassifier(),
    # LogisticRegression(),
    # LinearDiscriminantAnalysis(),
    # MLPClassifier(),
    # GradientBoostingClassifier() #likely to require approx. 30min of training time
]

results_df_groups = pd.DataFrame(columns=['removed_group', 'n_columns', 'model', 'acc', 'tpr'])

for key, value in feature_groups_map.items():

    df_trimmed = remove_columns_ints(df, value)

    results, models = train_models(models, df_trimmed)

    results_df_groups = results_df_groups.append({'removed_group': key, 'n_columns': df_trimmed.shape[1] - 4, 'model': results['model'][0], 'acc': results['acc'][0], 'tpr': results['tpr'][0]}, ignore_index=True)

results_df_groups


100%|██████████| 1/1 [00:09<00:00,  9.43s/it]


,removed_group,n_columns,model,acc,tpr
0,A_KEY_TIME_MEAN,91,RandomForestClassifier(),0.999117,0.999117
1,BOTH_DEV_C_COUNT,91,RandomForestClassifier(),0.999117,0.999117
2,GAP,83,RandomForestClassifier(),0.999117,0.999117
3,GAP_SD,91,RandomForestClassifier(),0.999411,0.999411
4,KEY_COUNT,91,RandomForestClassifier(),0.999411,0.999411
5,KEY_GAP_TIME,83,RandomForestClassifier(),0.998822,0.998822
6,KEY_TIME,82,RandomForestClassifier(),0.998528,0.998528
7,KEY_TIME_MEAN,91,RandomForestClassifier(),0.998822,0.998822
8,MOUSE_COUNT,91,RandomForestClassifier(),0.999411,0.999411
9,MOUSE_DDIR,90,RandomForestClassifier(),0.999411,0.999411


In [106]:
results_df_groups.to_csv(f'classification_results\\groups\\groups_removal.csv', index=False)

## Remaining only single groups

In [107]:
models = [
    # KNeighborsClassifier(3, weights="distance"),
    # GaussianNB(),
    # BernoulliNB(),
    # ComplementNB(),
    # MultinomialNB(),
    RandomForestClassifier(),
    # LogisticRegression(),
    # LinearDiscriminantAnalysis(),
    # MLPClassifier(),
    # GradientBoostingClassifier() #likely to require approx. 30min of training time
]

results_df_groups_remain = pd.DataFrame(columns=['removed_group', 'n_columns','model', 'acc', 'tpr'])

for key, value in feature_groups_map.items():

    df_trimmed = remain_columns_ints(df, value)

    results, models = train_models(models, df_trimmed)

    results_df_groups_remain = results_df_groups_remain.append({'removed_group': key, 'n_columns': df_trimmed.shape[1] - 4,  'model': results['model'][0], 'acc': results['acc'][0], 'tpr': results['tpr'][0]}, ignore_index=True)

results_df_groups_remain

100%|██████████| 1/1 [00:01<00:00,  1.15s/it]


,removed_group,n_columns,model,acc,tpr
0,A_KEY_TIME_MEAN,1,RandomForestClassifier(),0.224021,0.224021
1,BOTH_DEV_C_COUNT,1,RandomForestClassifier(),0.106270,0.106270
2,GAP,9,RandomForestClassifier(),0.989697,0.989697
3,GAP_SD,1,RandomForestClassifier(),0.059464,0.059464
4,KEY_COUNT,1,RandomForestClassifier(),0.012364,0.012364
5,KEY_GAP_TIME,9,RandomForestClassifier(),0.982632,0.982632
6,KEY_TIME,10,RandomForestClassifier(),0.957904,0.957904
7,KEY_TIME_MEAN,1,RandomForestClassifier(),0.203120,0.203120
8,MOUSE_COUNT,1,RandomForestClassifier(),0.025905,0.025905
9,MOUSE_DDIR,2,RandomForestClassifier(),0.591699,0.591699


In [108]:
results_df_groups_remain.to_csv(f'classification_results\\groups\\groups_removal_remain.csv', index=False)